# Downscaling the resolution of the Air Quality for different metropolis of Catalunya
Here we have the main script for the backend processing and prediction for the data.

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
